<a href="https://colab.research.google.com/github/zhang99111/DD2360HT22/blob/main/hw_3/hw3_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Next, we write a native CUDA code and save it as 'lab3_ex1.cu'


In [ ]:
%%writefile lab3_ex1.cu
#include <stdio.h>
#include <sys/time.h>
#include <stdlib.h>
#include <random>
#define DataType double

__global__ void vecAdd(DataType *in1, DataType *in2, DataType *out, int len) {
  //@@ Insert code to implement vector addition here
  int idx=0; 
  idx=threadIdx.x+blockDim.x * blockIdx.x;
  if(idx<len){
    out[idx]=in1[idx]+in2[idx];
  }

}
double cpuSecond() {
   struct timeval tp;
   gettimeofday(&tp,NULL);
   return ((double)tp.tv_sec + (double)tp.tv_usec*1.e-6);
}
//@@ Insert code to implement timer start

//@@ Insert code to implement timer stop

double stopTimer(double starttime) {
   struct timeval tp;
   gettimeofday(&tp,NULL);
   return ((double)tp.tv_sec + ((double)tp.tv_usec*1.e-6)-starttime);
}
int main(int argc, char **argv) {
  
  int inputLength;
  DataType *hostInput1;
  DataType *hostInput2;
  DataType *hostOutput;
  DataType *resultRef;
  DataType *deviceInput1;
  DataType *deviceInput2;
  DataType *deviceOutput;

  //@@ Insert code below to read in inputLength from args
  
  inputLength=atoi(argv[1]);
  printf("The input length is %d\n", inputLength);
  
  //@@ Insert code below to allocate Host memory for input and output
  
  hostInput1 = (DataType*)malloc(inputLength*sizeof(*hostInput1));
  hostInput2 = (DataType*)malloc(inputLength*sizeof(*hostInput2));
  hostOutput = (DataType*)malloc(inputLength*sizeof(*hostOutput));
  //@@ Insert code below to initialize hostInput1 and hostInput2 to random numbers, and create reference result in CPU
  std::uniform_real_distribution<DataType> distribution(0.0, 1.0);
  std::default_random_engine gen(1145);
  for (DataType *ptr : {hostInput1, hostInput2}) {
      for (int i=0; i<inputLength; ++i){
          ptr[i] = distribution(gen);
      }
  }
  resultRef = (DataType*)malloc(inputLength*sizeof(DataType));
  double starttime=cpuSecond();
  for(int i=0; i<inputLength; i++){
    resultRef[i]=hostInput1[i]+hostInput2[i];
  }
  double stoptime_cpu=stopTimer(starttime);

  //@@ Insert code below to allocate GPU memory here
  cudaMalloc(&deviceInput1, inputLength*sizeof(hostInput1));
  cudaMalloc(&deviceInput2, inputLength*sizeof(hostInput2));
  cudaMalloc(&deviceOutput, inputLength*sizeof(hostOutput));

  //@@ Insert code to below to Copy memory to the GPU here
  double starttime_memory=cpuSecond();
  cudaMemcpy(deviceInput1, hostInput1, inputLength*sizeof(DataType), cudaMemcpyHostToDevice);
  cudaMemcpy(deviceInput2, hostInput2, inputLength*sizeof(DataType), cudaMemcpyHostToDevice);
  cudaDeviceSynchronize();
  double stoptime_memory=stopTimer(starttime_memory);

   int Db =1024;
   int Dg = (inputLength + Db - 1) / Db;
  //@@ Initialize the 1D grid and block dimensions here
 
  
  //@@ Launch the GPU Kernel here
  double starttime_gpu=cpuSecond();
  vecAdd<<<Dg,Db>>>(deviceInput1,deviceInput2,deviceOutput,inputLength);
  cudaDeviceSynchronize();
  double stoptime_gpu=stopTimer(starttime_gpu);
  double starttime_memory_2=cpuSecond();

  //@@ Copy the GPU memory back to the CPU here
  cudaMemcpy(hostOutput, deviceOutput, inputLength*sizeof(DataType), cudaMemcpyDeviceToHost);
  cudaDeviceSynchronize();
  double stoptimer_memory_2=stopTimer(starttime_memory_2);


  //@@ Insert code below to compare the output with the reference
  for(int i=0;i<inputLength;i++)
  {
    if(resultRef[i] != hostOutput[i]&& abs(resultRef[i]-hostOutput[i])>0.0001){
      printf("error counted:%f",abs(resultRef[i]-hostOutput[i]));
      return 0;
    }
  }
  printf("cost Host->Device: %f - cost Device->Host: %f\n",stoptime_memory,stoptimer_memory_2);
  printf("CPU cost: %f - GPU cost: %f\n",stoptime_cpu,stoptime_gpu);
  //@@ Free the GPU memory here
  cudaFree(deviceInput1);
  cudaFree(deviceInput2);
  cudaFree(deviceOutput);


  //@@ Free the CPU memory here

  free(hostInput1);
  free(hostInput2);
  free(hostOutput);

  return 0;


  return 0;
}

Overwriting lab3_ex1.cu


## We compile the saved cuda code using nvcc compiler

In [ ]:
!nvcc -arch=sm_75 ./lab3_ex1.cu -o lab3_ex1



## Finally, we execute the binary of the compiled code

In [ ]:
!./lab3_ex1 1024

The input length is 1024
cost Host->Device: 0.000096 - cost Device->Host: 0.000023
CPU cost: 0.000006 - GPU cost: 0.000062


In [ ]:
!/usr/local/cuda-11/bin/nv-nsight-cu-cli ./lab3_ex1 1024

The input length is 1024
==PROF== Connected to process 470 (/content/lab3_ex1)
==PROF== Profiling "vecAdd" - 1: 0%....50%....100% - 8 passes
cost Host->Device: 0.000068 - cost Device->Host: 0.000068
CPU cost: 0.000007 - GPU cost: 0.757928
==PROF== Disconnected from process 470
[470] lab3_ex1@127.0.0.1
  vecAdd(double*, double*, double*, int), 2022-Dec-15 11:53:53, Context 1, Stream 7
    Section: GPU Speed Of Light
    ---------------------------------------------------------------------- --------------- ------------------------------
    DRAM Frequency                                                           cycle/nsecond                           4.94
    SM Frequency                                                             cycle/usecond                         594.49
    Elapsed Cycles                                                                   cycle                          3,064
    Memory [%]                                                                           %   

In [ ]:
!./lab3_ex1 32768

The input length is 32768
cost Host->Device: 0.000195 - cost Device->Host: 0.000215
CPU cost: 0.000247 - GPU cost: 0.000027


In [ ]:
!./lab3_ex1 65536

The input length is 65536
cost Host->Device: 0.000353 - cost Device->Host: 0.000429
CPU cost: 0.000460 - GPU cost: 0.000030


In [ ]:
!./lab3_ex1 131070

The input length is 131070
cost Host->Device: 0.000679 - cost Device->Host: 0.000825
CPU cost: 0.000914 - GPU cost: 0.000039


In [ ]:
!./lab3_ex1 262144

The input length is 262144
cost Host->Device: 0.001117 - cost Device->Host: 0.001524
CPU cost: 0.001756 - GPU cost: 0.000046


In [ ]:
!./lab3_ex1 524288

The input length is 524288
cost Host->Device: 0.002047 - cost Device->Host: 0.002832
CPU cost: 0.003772 - GPU cost: 0.000070


In [ ]:
!/usr/local/cuda-11/bin/nv-nsight-cu-cli ./lab3_ex1 131070

The input length is 131070
==PROF== Connected to process 511 (/content/lab3_ex1)
==PROF== Profiling "vecAdd" - 1: 0%....50%....100% - 8 passes
cost Host->Device: 0.000738 - cost Device->Host: 0.000855
CPU cost: 0.000898 - GPU cost: 0.719693
==PROF== Disconnected from process 511
[511] lab3_ex1@127.0.0.1
  vecAdd(double*, double*, double*, int), 2022-Dec-15 11:54:22, Context 1, Stream 7
    Section: GPU Speed Of Light
    ---------------------------------------------------------------------- --------------- ------------------------------
    DRAM Frequency                                                           cycle/nsecond                           4.88
    SM Frequency                                                             cycle/usecond                         585.85
    Elapsed Cycles                                                                   cycle                          8,407
    Memory [%]                                                                           % 

In [ ]:
!nvprof ./lab3_ex1 134217728

The input length is 134217728
tcmalloc: large alloc 1073741824 bytes == 0x56488a114000 @  0x7fc8a578f1e7 0x564888a076d1 0x7fc8a4422c87 0x564888a0748a
==523== NVPROF is profiling process 523, command: ./lab3_ex1 134217728
cost Host->Device: 0.463643 - cost Device->Host: 0.729260
CPU cost: 0.930963 - GPU cost: 0.012200
==523== Profiling application: ./lab3_ex1 134217728
==523== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   60.50%  727.87ms         1  727.87ms  727.87ms  727.87ms  [CUDA memcpy DtoH]
                   38.49%  463.12ms         2  231.56ms  230.84ms  232.28ms  [CUDA memcpy HtoD]
                    1.01%  12.145ms         1  12.145ms  12.145ms  12.145ms  vecAdd(double*, double*, double*, int)
      API calls:   81.46%  1.19279s         3  397.60ms  231.09ms  729.25ms  cudaMemcpy
                   17.23%  252.27ms         3  84.091ms  1.1112ms  250.05ms  cudaMalloc
                    0.84%  12.241ms  